In [16]:
import pandas as pd
import numpy as np
import joblib
import ast
from joblib import Parallel, delayed
import json
import os
from tqdm import tqdm

In [17]:
item_data = pd.read_csv('/opt/ml/wine/data/wine_df.csv')

In [18]:
def drop_columns(df):
    to_drop = ['Red Fruit','Tropical','Tree Fruit','Oaky',
               'Ageing','Black Fruit','Citrus','Dried Fruit','Earthy',
               'Floral','Microbio','Spices', 'Vegetal',
               'Unnamed: 58', 'None_child', 'None_count', "None",
                'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64']

    for c in to_drop:
        try:
            df.drop(c, axis = 1, inplace= True)  
        except Exception as e: 
            print(e)
    
    return df

def fill_na(df):
    with open('/opt/ml/wine/code/data/meta_data/string_columns.json','r',encoding='utf-8') as f:  
        col = json.load(f)
        df[col] = df[col].fillna('')
    with open('/opt/ml/wine/code/data/meta_data/dict_columns.json','r',encoding='utf-8') as f:  
        col = json.load(f)
        df[col] = df[col].fillna('{}')
    with open('/opt/ml/wine/code/data/meta_data/seq_columns.json','r',encoding='utf-8') as f:  
        col = json.load(f)
        df[col] = df[col].fillna("['None']")

    with open('/opt/ml/wine/code/data/meta_data/float_columns.json','r',encoding='utf-8') as f:  
        col = json.load(f)
        #col = [c for c in col if '_count' in c]
        for c in col:
            if c in df.columns: df[c] = df[c].fillna(0)
    

    return df


In [19]:
item_data = fill_na(item_data)
item_data = drop_columns(item_data)

"['None_child'] not found in axis"
"['None'] not found in axis"


In [20]:
item_data.isnull().sum()

url                  0
country              0
region               0
winery               0
winetype             0
grape                0
name                 0
vintage              0
house                0
price                0
rating               0
num_votes            0
star5                0
star4                0
star3                0
star2                0
star1                0
pairing              0
wine_style           0
Red Fruit_count      0
Red_Fruit_child      0
Tropical_count       0
Tropical_child       0
Tree Fruit_count     0
Tree_Fruit_child     0
Oaky_count           0
Oaky_child           0
Ageing_child         0
Ageing_count         0
Black Fruit_count    0
Black_Fruit_child    0
Citrus_child         0
Citrus_count         0
Dried Fruit_count    0
Dried_Fruit_child    0
Earthy_child         0
Earthy_count         0
Floral_child         0
Floral_count         0
Microbio_child       0
Microbio_count       0
Spices_child         0
Spices_count         0
Vegetal_chi

In [5]:
notes = ['Red Fruit','Tropical','Tree Fruit','Oaky',
        'Ageing','Black Fruit','Citrus','Dried Fruit','Earthy',
        'Floral','Microbio','Spices', 'Vegetal']


In [6]:
def str2list(x):
    if len(x) > 0:
        if x[0] != '[':
            list = [x]
        else: 
            list = ast.literal_eval(x)
    else: list = []
    
    return list

def feature_mapper(df, column):
    unique_val = df[column].unique()
    feature2idx = {f:i for i, f in enumerate(unique_val)}
    idx2feature = {i:f for i, f in enumerate(unique_val)}

    if not os.path.exists('/opt/ml/wine/code/meta_data/'): 
        os.makedirs('/opt/ml/wine/code/meta_data/')

    with open(f'/opt/ml/wine/code/feature_map/{column}2idx.json','w',encoding='utf-8') as f:  
        json.dump(feature2idx, f, ensure_ascii=False)
    with open(f'/opt/ml/wine/code/feature_map/idx2{column}.json','w',encoding='utf-8') as f:  
        json.dump(idx2feature, f, ensure_ascii=False)

    return feature2idx, idx2feature

def list_feature_mapper(df, column):

    df[column] = df[column].apply(lambda x: str2list(x))

    exploded = item_data[column].explode(column)
    unique_val = set(list(exploded))
    feature_dic = {}

    feature2idx = {f:i for i, f in enumerate(unique_val)}
    idx2feature = {i:f for i, f in enumerate(unique_val)}

    if not os.path.exists('/opt/ml/wine/code/meta_data/'): 
        os.makedirs('/opt/ml/wine/code/meta_data/')

    with open(f'/opt/ml/wine/code/feature_map/{column}2idx.json','w',encoding='utf-8') as f:  
        json.dump(feature2idx, f, ensure_ascii=False)
    with open(f'/opt/ml/wine/code/feature_map/idx2{column}.json','w',encoding='utf-8') as f:  
        json.dump(idx2feature, f, ensure_ascii=False)

    return df, feature2idx, idx2feature

def map_all_single_features(df):
    single_category_columns = ['country', 'region', 'winery', 'winetype', 'vintage', 'house', 'wine_style']
    for c in single_category_columns:
        feature_mapper(df, c)
    return  

def map_all_list_features(df):
    list_columns = ['grape','pairing']
    for c in list_columns:
        df ,_ ,_ = list_feature_mapper(df, c)
    return df 


def note_mapper(df, note_col):

    note = note_col

    note_col = note_col + '_child'
    note_col = note_col.replace(' ','_')
    
    try:
        df[note_col] = df[note_col].apply(lambda x: ast.literal_eval(x))
    except Exception as e:
        print(e)
    
    unique_val = []
    for note_dic in df[note_col]:
        unique_val.extend(list(note_dic.keys()))
    unique_val = list(set(unique_val))

    feature2idx = {f:i for i, f in enumerate(unique_val)}
    idx2feature = {i:f for i, f in enumerate(unique_val)}

    if not os.path.exists('/opt/ml/wine/code/meta_data/'): 
        os.makedirs('/opt/ml/wine/code/meta_data/')

    with open(f'/opt/ml/wine/code/feature_map/{note}2idx.json','w',encoding='utf-8') as f:  
        json.dump(feature2idx, f, ensure_ascii=False)
    with open(f'/opt/ml/wine/code/feature_map/idx2{note}.json','w',encoding='utf-8') as f:  
        json.dump(idx2feature, f, ensure_ascii=False)

    return feature2idx, idx2feature

def expand_notes(df):
    notes = ['Red Fruit','Tropical','Tree Fruit','Oaky',
        'Ageing','Black Fruit','Citrus','Dried Fruit','Earthy',
        'Floral','Microbio','Spices', 'Vegetal']
    
    i = 0
    for note_col in tqdm(notes):

        note_df = []

        feature2idx, idx2feature = note_mapper(df, note_col)

        for note_dic in tqdm(df[note_col.replace(' ','_') + '_child']):
            row_data = [0 for i in range(len(feature2idx))]

            for note in note_dic:
                row_data[feature2idx[note]] = note_dic[note]
  
            note_df.append(row_data)
        
        columns = [idx2feature[i] for i in range(len(idx2feature))]
        note_df = pd.DataFrame(note_df, columns=columns)

        if i == 0:
            result = note_df
            i += 1
        else:
            result = pd.concat([result, note_df], axis=1)

        df.drop(note_col.replace(' ','_') + '_child', axis = True, inplace = True)

    df = pd.concat([df, result], axis=1)
    return df
  

In [7]:
map_all_single_features(item_data)
item_data = map_all_list_features(item_data)
item_data = expand_notes(item_data)

100%|██████████| 13/13 [00:46<00:00,  3.56s/it]


In [8]:
item_data.to_csv('/opt/ml/wine/data/item_data.csv', encoding='utf-8-sig')

In [3]:
item_data = pd.read_csv('/opt/ml/wine/data/wine_df.csv')
review_data = pd.read_csv('/opt/ml/wine/data/review_df.csv')


In [6]:
review_data = review_data[review_data['user_url'].isna()== False]

In [8]:
review_data['date'] = review_data['date'].apply(lambda x: pd.to_datetime(x))

/tmp/ipykernel_106682/2395739959.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_data['date'] = review_data['date'].apply(lambda x: pd.to_datetime(x))


In [11]:
review_data2 = pd.read_csv('/opt/ml/wine/data/[김영서]review_df.csv')
review_data3 = pd.read_csv('/opt/ml/wine/data/[박재성]review_df.csv')

In [13]:
review_data = pd.concat([review_data, review_data3], axis = 0)

In [14]:
review_data.to_csv('/opt/ml/wine/data/review_df.csv', encoding='utf-8-sig')

In [3]:
def note_mapper(df, note_col):

    note = note_col

    note_col = note_col + '_child'
    note_col = note_col.replace(' ','_')
    
    try:
        df[note_col] = df[note_col].apply(lambda x: ast.literal_eval(x))
    except Exception as e:
        print(e)
    
    unique_val = []
    for note_dic in df[note_col]:
        unique_val.extend(list(note_dic.keys()))
    unique_val = list(set(unique_val))

    feature2idx = {f:i for i, f in enumerate(unique_val)}
    idx2feature = {i:f for i, f in enumerate(unique_val)}

    if not os.path.exists('/opt/ml/wine/code/data/feature_map/'): 
        os.makedirs('/opt/ml/wine/code/data/feature_map/')

    with open(f'/opt/ml/wine/code/data/feature_map/{note}2idx.json','w',encoding='utf-8') as f:  
        json.dump(feature2idx, f, ensure_ascii=False)
    with open(f'/opt/ml/wine/code/data/feature_map/idx2{note}.json','w',encoding='utf-8') as f:  
        json.dump(idx2feature, f, ensure_ascii=False)

    return feature2idx, idx2feature

In [37]:
def expand_notes(df):
    notes = ['Red Fruit','Tropical','Tree Fruit','Oaky',
        'Ageing','Black Fruit','Citrus','Dried Fruit','Earthy',
        'Floral','Microbio','Spices', 'Vegetal']
    
    i = 0
    for note_col in tqdm(notes):

        note_df = []

        feature2idx, idx2feature = note_mapper(df, note_col)

        for note_dic in tqdm(df[note_col.replace(' ','_') + '_child']):
            row_data = [0 for i in range(len(feature2idx))]

            for note in note_dic:
                row_data[feature2idx[note]] = note_dic[note]
  
            note_df.append(row_data)
        
        columns = [idx2feature[i] for i in range(len(idx2feature))]
        note_df = pd.DataFrame(note_df, columns=columns, index = df.index)

        if i == 0:
            result = note_df
            i += 1
        else:
            result = pd.concat([result, note_df], axis=1)
        df.drop(note_col.replace(' ','_') + '_child', axis = True, inplace = True)

    df = pd.concat([df, result], axis=1)
    return df
  

In [38]:
df_chunks = np.array_split(item_data, 10)

In [39]:
d1 = expand_notes(df_chunks[0])
d2 = expand_notes(df_chunks[1])
d3 = expand_notes(df_chunks[2])

  0%|          | 0/13 [00:00<?, ?it/s]

malformed node or string: {'cherry': 4, 'raspberry': 1, 'red fruit': 1}


  0%|          | 0/13 [00:00<?, ?it/s]

malformed node or string: {'watermelon': 1, 'cranberry': 1}


  0%|          | 0/13 [00:00<?, ?it/s]

malformed node or string: {}


100%|██████████| 13/13 [00:05<00:00,  2.59it/s]


In [25]:
d3

,url,country,region,winery,winetype,grape,name,vintage,house,price,...,chard,fresh-cut grass,celery,wheat grass,gooseberry,tomato,asparagus,chayote squash,pea shoot,radicchio
51890,https://www.vivino.com/US-CA/en/bagnol-cassis/...,France,Cassis,Bagnol,White wine,Clairette,Cassis 2019,2019.0,Bagnol,37.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51891,https://www.vivino.com/US-CA/en/la-mission-hau...,France,Pessac-Léognan,Château La Mission Haut-Brion,Red wine,Merlot,La Chapelle de La Mission Haut-Brion Pessac-Lé...,2010.0,Château La Mission Haut-Brion,204.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51892,https://www.vivino.com/US-CA/en/es-gonzalez-by...,Spain,Jerez-Xérès-Sherry,Gonzalez-Byass,Fortified Wine,Palomino,Solera 1847 Cream N.V.,NaN,Gonzalez-Byass,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51893,https://www.vivino.com/US-CA/en/simon-bize-and...,France,Bourgogne,Simon Bize & Fils,White wine,Chardonnay,Bourgogne 'Les Perrières' Blanc 2018,2018.0,Simon Bize & Fils,53.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51894,https://www.vivino.com/US-CA/en/john-duval-win...,Australia,Barossa Valley,John Duval,Red wine,Shiraz/Syrah,Eligo Shiraz 2014,2014.0,John Duval,89.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25942,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [40]:
dd = pd.concat([d1, d2, d3], axis = 0)

In [45]:
len(d1) + len(d2) + len(d3)

23352

In [46]:
dd

,url,country,region,winery,winetype,grape,name,vintage,house,price,...,fresh-cut grass,celery,wheat grass,gooseberry,tomato,asparagus,pea shoot,radicchio,peppercress,dried red chili
0,https://www.vivino.com/US-CA/en/tour-de-biot-b...,France,Bordeaux,Château Tour de Biot,Red wine,"['Merlot', 'Cabernet Sauvignon', 'Cabernet Fra...",Bordeaux 2018,2018.0,Château Tour de Biot,18.99,...,0,0,0,0,0,0,0,0,NaN,NaN
1,https://www.vivino.com/US-CA/en/pio-cesare-mos...,Italy,Moscato d'Asti,Pio Cesare,Sparkling wine,Moscato Bianco,Moscato d'Asti 2020,2020.0,Pio Cesare,21.99,...,0,0,0,0,0,0,0,0,NaN,NaN
2,https://www.vivino.com/US-CA/en/fr-g-descombes...,France,Régnié,G. Descombes,Red wine,Gamay,Régnié 2020,2020.0,G. Descombes,31.99,...,0,0,0,0,5,0,0,0,NaN,NaN
3,https://www.vivino.com/US-CA/en/ava-grace-rose...,United States,California,AVA Grace,Rosé wine,Pinot Gris,Rosé 2019,2019.0,AVA Grace,8.99,...,0,0,0,0,0,0,0,0,NaN,NaN
4,https://www.vivino.com/US-CA/en/louis-dumont-b...,France,Champagne,Louis Dumont,Sparkling wine,"['Chardonnay', 'Pinot Noir', 'Pinot Meunier']",Brut Rosé Champagne N.V.,NaN,Louis Dumont,NaN,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23347,https://www.vivino.com/US-CA/en/domaine-courbi...,France,Saint-Péray,Domaine Courbis,White wine,"['Marsanne', 'Roussanne']",Saint Péray Le Tram 2021,2021.0,Domaine Courbis,22.51,...,0,0,0,0,0,0,0,0,0.0,0.0
23348,https://www.vivino.com/US-CA/en/tenuta-delle-t...,Italy,Etna,Tenuta delle Terre Nere,White wine,"['Nerello Mascalese', 'Nerello Cappuccio']",Etna Calderara Sottana Bianco 2019,2019.0,Tenuta delle Terre Nere,30.32,...,0,0,0,0,1,0,0,0,0.0,0.0
23349,https://www.vivino.com/US-CA/en/chateau-de-bea...,France,Châteauneuf-du-Pape,Château de Beaucastel,Red wine,"['Shiraz/Syrah', 'Counoise', 'Cinsault', 'Gren...",Châteauneuf-du-Pape 1995,1995.0,Château de Beaucastel,129.95,...,0,1,0,2,26,3,0,0,0.0,0.0
23350,https://www.vivino.com/US-CA/en/castellani-tou...,Italy,Maremma Toscana,Castellani Touton,Red wine,Sangiovese,Monsalaia Maremma Toscana 2019,2019.0,Castellani Touton,12.20,...,0,0,0,0,1,0,0,0,0.0,0.0
